In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
# Criação da sessão Spark
spark = SparkSession.builder \
    .appName("PySpark BigQuery Connection") \
    .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.23.2') \
    .config("spark.jars", "/usr/local/lib/spark-connectors/bigquery-connector-hadoop2-latest.jar") \
    .config("spark.driver.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true -Dio.netty.noUnsafe=true") \
    .config("spark.executor.extraJavaOptions", "-Dio.netty.tryReflectionSetAccessible=true -Dio.netty.noUnsafe=true") \
    .getOrCreate()

24/09/28 14:22:18 WARN Utils: Your hostname, spark-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/09/28 14:22:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-315bab2c-9ea1-47cd-8b66-2540eda9f648;1.0
	confs: [default]
	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 in central
:: resolution report :: resolve 903ms :: artifacts dl 22ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.23.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	------------------------------------------------------------------

In [3]:
spark.conf.set("viewsEnabled", True)
spark.conf.set("materializationDataset", "SOR")

In [4]:
sc = spark.sparkContext
sc.setLogLevel("INFO")
sc._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
sc._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.json.keyfile', '/usr/local/lib/gcp/credentials/my-project-1508437523553-e9bafe7e3368.json')

In [8]:
# Função para salvar DataFrame em formato Parquet
def save_to_bigquery(df, dataset, table_name):
    # Salva o DataFrame em formato Parquet
    df.write \
    .format("bigquery") \
    .option("table", f"{dataset.upper()}.{table_name}") \
    .option("temporaryGcsBucket", "meu-bucket-temporario-spark") \
    .option("credentialsFile", "/usr/local/lib/gcp/credentials/my-project-1508437523553-e9bafe7e3368.json") \
    .mode("overwrite") \
    .save()

In [5]:
# Função para ler dados do BigQuery
def read_from_bigquery(dataset, table_name):  
    df = spark.read \
        .format('bigquery') \
        .option('table', f"{dataset.upper()}.{table_name}") \
        .option("credentialsFile", "/usr/local/lib/gcp/credentials/my-project-1508437523553-e9bafe7e3368.json") \
        .load()

    return df

In [ ]:
# # Lista de tabelas do BigQuery
# A001B3|2650459       |
# |A001B1|2650459       |
# |A001B2|2650459       |
# |UF    |2650459       |
# |Ano   |2650459       |
# |V1012 |2650459       |
# |A002  |2650459       |
# |V1016 |2650459       |
# |A003  |2650459       |
# |UPA   |2650459       |
# |A004  |2650459       |
# |V1023 |2650459       |
# |A005  |2650459       |
# |V1031 |2650459       |
# |posest|2650459       |
# |B0018 |2650459       |
# |A001  |2650459       |
# |B0017 |2650459       |
# |A001A |2650459       |
# |V1013 |2650459  

In [75]:
df_dimensao = read_from_bigquery('SOR', 'tbx002_dimensao_geral')
df_dimensao.createOrReplaceTempView("tbx002_dimensao_geral")

In [76]:
df = read_from_bigquery('SOR', 'tbx001_data')

In [77]:
# Dicionário de mapeamento de renomeação de colunas
col_rename_map = {
    "UF": "uf",
    "Ano": "ano",
    "V1013": "mes",
    "V1012": "semana",
    "A001B3": "ano_nascimento",
    "A003": "sexo",
    "A004": "cor_raca",
    "V1023": "tipo_area",
    "A005": "escolaridade",
    "B0011": "teve_febre",
    "B0014": "teve_dificuldade_respirar",
    "B0015": "teve_dor_cabeca",
    "B0019": "teve_fadiga",
    "B00111": "teve_perda_cheiro",
    "B002": "foi_posto_saude",
    "B0031": "ficou_em_casa",
    "B005": "ficou_internado",
    "B009B": "resultado_covid",
    "B007": "tem_plano_saude",
    "C01011": "faixa_rendimento",
    "F001": "situacao_domicilio"
}

# Aplicar a renomeação das colunas e selecionar apenas as colunas renomeadas
df = df.select([F.col(old_name).alias(new_name) for old_name, new_name in col_rename_map.items()])


In [78]:
df.createOrReplaceTempView("tbx001_data")

In [97]:
# realizar o join entre os dataframes, dimensionando a tabela de fatos
query = """
SELECT 
T2.categoria_descricao AS uf,
T1.ano,
T1.mes,
T1.semana,
T1.ano_nascimento,
T3.categoria_descricao AS sexo,
T4.categoria_descricao AS cor_raca,
T5.categoria_descricao AS tipo_area,
T6.categoria_descricao AS escolaridade,
T7.categoria_descricao AS teve_febre,
T8.categoria_descricao AS teve_dificuldade_respirar,
T9.categoria_descricao AS teve_dor_cabeca,
T10.categoria_descricao AS teve_fadiga,
T11.categoria_descricao AS teve_perda_cheiro,
T12.categoria_descricao AS foi_posto_saude,
T13.categoria_descricao AS ficou_em_casa,
T14.categoria_descricao AS ficou_internado,
T15.categoria_descricao AS resultado_covid,
T16.categoria_descricao AS tem_plano_saude,
T17.categoria_descricao AS faixa_rendimento,
T18.categoria_descricao AS situacao_domicilio
FROM tbx001_data T1
LEFT JOIN tbx002_dimensao_geral T2 on T2.codigo_variavel = 'UF' and T1.uf = T2.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T3 on T3.codigo_variavel = 'A003' and T1.sexo = T3.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T4 on T4.codigo_variavel = 'A004' and T1.cor_raca = T4.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T5 on T5.codigo_variavel = 'V1023' and T1.tipo_area = T5.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T6 on T6.codigo_variavel = 'A005' and T1.escolaridade = T6.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T7 on T7.codigo_variavel = 'B0011' and T1.teve_febre = T7.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T8 on T8.codigo_variavel = 'B0014' and T1.teve_dificuldade_respirar = T8.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T9 on T9.codigo_variavel = 'B0015' and T1.teve_dor_cabeca = T9.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T10 on T10.codigo_variavel = 'B0019' and T1.teve_fadiga = T10.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T11 on T11.codigo_variavel = 'B00111' and T1.teve_perda_cheiro = T11.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T12 on T12.codigo_variavel = 'B002' and T1.foi_posto_saude = T12.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T13 on T13.codigo_variavel = 'B0031' and T1.ficou_em_casa = T13.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T14 on T14.codigo_variavel = 'B005' and T1.ficou_internado = T14.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T15 on T15.codigo_variavel = 'B009B' and T1.resultado_covid = T15.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T16 on T16.codigo_variavel = 'B007' and T1.tem_plano_saude = T16.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T17 on T17.codigo_variavel = 'C01011' and T1.faixa_rendimento = T17.categoria_tipo
LEFT JOIN tbx002_dimensao_geral T18 on T18.codigo_variavel = 'F001' and T1.situacao_domicilio = T18.categoria_tipo

"""

In [98]:
# Executar a consulta SQL
df_joined = spark.sql(query)

In [100]:
df_joined.printSchema()

root
 |-- uf: string (nullable = true)
 |-- ano: string (nullable = true)
 |-- mes: string (nullable = true)
 |-- semana: string (nullable = true)
 |-- ano_nascimento: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cor_raca: string (nullable = true)
 |-- tipo_area: string (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- teve_febre: string (nullable = true)
 |-- teve_dificuldade_respirar: string (nullable = true)
 |-- teve_dor_cabeca: string (nullable = true)
 |-- teve_fadiga: string (nullable = true)
 |-- teve_perda_cheiro: string (nullable = true)
 |-- foi_posto_saude: string (nullable = true)
 |-- ficou_em_casa: string (nullable = true)
 |-- ficou_internado: string (nullable = true)
 |-- resultado_covid: string (nullable = true)
 |-- tem_plano_saude: string (nullable = true)
 |-- faixa_rendimento: string (nullable = true)
 |-- situacao_domicilio: string (nullable = true)



In [99]:
df_joined.show()

24/09/28 15:51:44 INFO DirectBigQueryRelation: |Querying table my-project-1508437523553.SOR.tbx001_data, parameters sent from Spark:|requiredColumns=[UF,Ano,V1013,V1012,A001B3,A003,A004,V1023,A005,B0011,B0014,B0015,B0019,B00111,B002,B0031,B005,B009B,B007,C01011,F001],|filters=[]
24/09/28 15:51:45 INFO ReadSessionCreator: Read session:{"readSessionName":"projects/my-project-1508437523553/locations/us-east1/sessions/CAISDDNCc1IwVUtlRExMORoCdngaAnVo","readSessionCreationStartTime":"2024-09-28T18:51:44.151Z","readSessionCreationEndTime":"2024-09-28T18:51:45.084Z","readSessionPrepDuration":414,"readSessionCreationDuration":519,"readSessionDuration":933}
24/09/28 15:51:45 INFO ReadSessionCreator: Requested 20000 max partitions, but only received 4 from the BigQuery Storage API for session projects/my-project-1508437523553/locations/us-east1/sessions/CAISDDNCc1IwVUtlRExMORoCdngaAnVo. Notice that the number of streams in actual may be lower than the requested number, depending on the amount pa

+--------+----+---+------+--------------+------+--------+---------+--------------------+----------+-------------------------+---------------+-----------+-----------------+---------------+-------------+---------------+---------------+---------------+----------------+--------------------+
|      uf| ano|mes|semana|ano_nascimento|  sexo|cor_raca|tipo_area|        escolaridade|teve_febre|teve_dificuldade_respirar|teve_dor_cabeca|teve_fadiga|teve_perda_cheiro|foi_posto_saude|ficou_em_casa|ficou_internado|resultado_covid|tem_plano_saude|faixa_rendimento|  situacao_domicilio|
+--------+----+---+------+--------------+------+--------+---------+--------------------+----------+-------------------------+---------------+-----------+-----------------+---------------+-------------+---------------+---------------+---------------+----------------+--------------------+
|Rondônia|2020| 07|     3|          1958| Homem|   Parda|  Capital|       Sem instrução|      Não |                     Não |           

24/09/28 15:52:01 INFO Executor: Finished task 0.0 in stage 383.0 (TID 389). 3651 bytes result sent to driver
24/09/28 15:52:01 INFO TaskSetManager: Finished task 0.0 in stage 383.0 (TID 389) in 759 ms on 10.0.2.15 (executor driver) (1/1)
24/09/28 15:52:01 INFO TaskSchedulerImpl: Removed TaskSet 383.0, whose tasks have all completed, from pool 
24/09/28 15:52:01 INFO DAGScheduler: ResultStage 383 (showString at <unknown>:0) finished in 0,834 s
24/09/28 15:52:01 INFO DAGScheduler: Job 236 is finished. Cancelling potential speculative or zombie tasks for this job
24/09/28 15:52:01 INFO TaskSchedulerImpl: Killing all running tasks in stage 383: Stage finished
24/09/28 15:52:01 INFO DAGScheduler: Job 236 finished: showString at <unknown>:0, took 0,863838 s


24/09/28 15:52:33 INFO BlockManagerInfo: Removed broadcast_302_piece0 on 10.0.2.15:38963 in memory (size: 295.0 B, free: 366.2 MiB)
24/09/28 15:52:33 INFO BlockManagerInfo: Removed broadcast_293_piece0 on 10.0.2.15:38963 in memory (size: 359.0 B, free: 366.2 MiB)
24/09/28 15:52:33 INFO BlockManagerInfo: Removed broadcast_307_piece0 on 10.0.2.15:38963 in memory (size: 430.0 B, free: 366.2 MiB)
24/09/28 15:52:33 INFO BlockManagerInfo: Removed broadcast_291_piece0 on 10.0.2.15:38963 in memory (size: 215.0 B, free: 366.2 MiB)
24/09/28 15:52:33 INFO BlockManagerInfo: Removed broadcast_308_piece0 on 10.0.2.15:38963 in memory (size: 18.8 KiB, free: 366.2 MiB)
24/09/28 15:52:34 INFO BlockManagerInfo: Removed broadcast_297_piece0 on 10.0.2.15:38963 in memory (size: 295.0 B, free: 366.2 MiB)
24/09/28 15:52:34 INFO BlockManagerInfo: Removed broadcast_301_piece0 on 10.0.2.15:38963 in memory (size: 467.0 B, free: 366.2 MiB)
24/09/28 15:52:34 INFO BlockManagerInfo: Removed broadcast_305_piece0 on 10

In [101]:
save_to_bigquery(df_joined, "SOT", "tbx001_data")

24/09/28 15:57:29 INFO GhfsStorageStatistics: Detected potential high latency for operation op_get_file_status. latencyMs=2169; previousMaxLatencyMs=0; operationCount=1; context=gs://meu-bucket-temporario-spark/.spark-bigquery-local-1727544150129-0a11234e-5570-4d0e-b2cd-951e11d2ceef
24/09/28 15:57:29 INFO DirectBigQueryRelation: |Querying table my-project-1508437523553.SOR.tbx001_data, parameters sent from Spark:|requiredColumns=[UF,Ano,V1013,V1012,A001B3,A003,A004,V1023,A005,B0011,B0014,B0015,B0019,B00111,B002,B0031,B005,B009B,B007,C01011,F001],|filters=[]
24/09/28 15:57:30 INFO ReadSessionCreator: Read session:{"readSessionName":"projects/my-project-1508437523553/locations/us-east1/sessions/CAISDHpsZEFTZEUzU004LRoCdngaAnVo","readSessionCreationStartTime":"2024-09-28T18:57:29.883Z","readSessionCreationEndTime":"2024-09-28T18:57:30.642Z","readSessionPrepDuration":234,"readSessionCreationDuration":525,"readSessionDuration":759}
24/09/28 15:57:30 INFO ReadSessionCreator: Requested 20000 